### 練習問題2.3
平⾯上の⻑⽅形の表現を実装せよ  
(ヒント：練習問題 2.2を利⽤するといいかもしれない)。  
そのコンストラクタとセレクタを使って、  
ある⻑⽅形の外周の⻑さと⾯積を計算する⼿続きを書け。  
次に、⻑⽅形の異なる表現を実装せよ。  
適切な抽象化の壁を使ってシステムを設計し、  
外周と⾯積を求める同じ⼿続きがどちらの表現によっても動くようにできるだろうか。

In [1]:
; 点のコンストラクタ
(define (make-point x y)(cons x y))
; セレクタ
(define (x-point p)(car p))
(define (y-point p)(cdr p))
(define (print-point p)
    (newline)
    (display "(")
    (display (x-point p))
    (display ",")
    (display (y-point p))
    (display ")"))

; 線分コンストラクタ
(define (make-segment p1 p2)(cons p1 p2))
; セレクタ
(define (start-segment s)(car s))
(define (end-segment s)(cdr s))
(define (midpoint-segment s)
  (let ((p1 (start-segment s))
        (p2 (end-segment s)))
    (let ((x1 (x-point p1))
          (y1 (y-point p1))
          (x2 (x-point p2))
          (y2 (y-point p2)))
      (make-point (+ (/ (- x2 x1) 2) x1) (+ (/ (- y2 y1) 2) y1))
      )
    )
  )

(define (make-rect p1 p2)(make-segment p1 p2))
; セレクタ
(define (point-rect pos r)
  (let ((p1 (start-segment r))
        (p2 (end-segment r)))
    (let ((x1 (x-point p1))
          (y1 (y-point p1))
          (x2 (x-point p2))
          (y2 (y-point p2)))
        (
         cond ((= pos 0) (make-point (min x1 x2) (max y1 y2)))  ; top-left
              ((= pos 1) (make-point (max x1 x2) (max y1 y2)))  ; top-right
              ((= pos 2) (make-point (max x1 x2) (min y1 y2)))  ; bottom-right
              ((= pos 3) (make-point (min x1 x2) (min y1 y2)))  ; bottom-left
              (else (error "pos error." ()))
         )
      )
    )
  )
(define (width-rect r)
  (let ((p1 (point-rect 0 r))
        (p2 (point-rect 1 r)))
     (- (x-point p2) (x-point p1))
    )
  )
(define (height-rect r)
  (let ((p1 (point-rect 3 r))
        (p2 (point-rect 0 r)))
     (- (y-point p2) (y-point p1))
    )
  )

; 面積
(define (area-rect r)
  (let ((width (width-rect r))
        (height (height-rect r)))
        (* width height)
    )
  )
; 外周
(define (perimeter-rect r)
  (let ((width (width-rect r))
        (height (height-rect r)))
        (+ (* width 2) (* height 2))
    )
  )



(define p1 (make-point 1 2))
(print-point p1)
(define p2 (make-point 5 4))
(print-point p2)
(define p3 (midpoint-segment (make-segment p1 p2)))
(print-point p3)

(newline)
(define r (make-rect p1 p2))
(display (width-rect r))
(newline)
(display (height-rect r))
(newline)
(display (area-rect r))
(newline)
(display (perimeter-rect r))
(newline)




(1,2)
(5,4)
(3,3)
4
2
8
12


In [2]:
; コンストラクタ
(define (make-rect center width height)
    (let ((half_width (/ width 2))
          (half_height (/ height 2)))
      (let ((x1 (- (x-point center) half_width))
            (y1 (- (y-point center) half_height))
            (x2 (+ (x-point center) half_width))
            (y2 (+ (y-point center) half_height)))
        (let ((p1 (make-point x1 y1))
              (p2 (make-point x2 y2)))
          (make-segment p1 p2)
        )
      )
    )
  )

(define p1 (make-point 1 2))
(print-point p1)
(define p2 (make-point 5 4))
(print-point p2)

(newline)
(define r (make-rect (make-point 3 3) 4 2))
(display (width-rect r))
(newline)
(display (height-rect r))
(newline)
(display (area-rect r))
(newline)
(display (perimeter-rect r))
(newline)


(1,2)
(5,4)
4
2
8
12
